### Import libraries

In [15]:
# Import necessary libraries
import os

# Import necessary libraries for data processing
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Import necessary libraries for model
from keras.callbacks import Callback
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from keras.metrics import Precision, Recall, F1Score, AUC

# Import necessary libraries for evaluation
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


## Directories

In [4]:
market_data_1_dir = 'market_data_A_0.csv'
market_data_2_dir = 'market_data_A_1.csv'

trade_data_dir = 'trade_data__A.csv'

Save_model_dir = 'LSTM_model'

# Data Preparation

### Load and merge data

In [7]:
# Load the data from a CSV file
market_data_1 = pd.read_csv(market_data_1_dir)
market_data_2 = pd.read_csv(market_data_2_dir)
market_data = pd.concat([market_data_1, market_data_2])

# Load the trade data from a CSV file
trade_data = pd.read_csv(trade_data_dir)

# Handle the null values
market_data.dropna(subset=['timestamp'], inplace=True)
trade_data.dropna(subset=['timestamp'], inplace=True)

# Convert the 'timestamp' column into datetime
market_data['timestamp'] = pd.to_datetime(market_data['timestamp'])
trade_data['timestamp'] = pd.to_datetime(trade_data['timestamp'])

# merge the data on the nearest timestamp
merged_data = pd.merge_asof(
    market_data.sort_values('timestamp'),
    trade_data.sort_values('timestamp'),
    by='timestamp',
    direction='nearest')

<ipython-input-7-7d60c1172efa>:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  market_data['timestamp'] = pd.to_datetime(market_data['timestamp'])
<ipython-input-7-7d60c1172efa>:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trade_data['timestamp'] = pd.to_datetime(trade_data['timestamp'])


### Feature Engineering

In [8]:
# Calculate the spread between the market and trade data
merged_data['spread'] = merged_data['askPrice'] - merged_data['bidPrice']

# Add a moving average of the spread
merged_data['bidPrice_ma'] = merged_data['bidPrice'].rolling(window=5).mean()
merged_data['askPrice_ma'] = merged_data['askPrice'].rolling(window=5).mean()
merged_data.fillna(method='bfill', inplace=True)

<ipython-input-8-4270b056c046>:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='bfill', inplace=True)
<ipython-input-8-4270b056c046>:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_data.fillna(method='bfill', inplace=True)


### Nomalise Features

In [9]:
# Normalize the data to a range of 0 to 1
scaler = MinMaxScaler()
features = ['bidVolume', 'bidPrice', 'askVolume', 'askPrice', 'spread', 'bidPrice_ma', 'askPrice_ma']
scaler.fit(merged_data[features])

MinMaxScaler()

### Define input and output

In [10]:
# Define timesteps
timesteps = 10

# Prepare input (x) and output (y)
x = []
y = []

for i in range(len(merged_data) - timesteps):
    x.append(merged_data[features].iloc[i:i + timesteps].values)
    y.append(merged_data['bidPrice'].iloc[i + timesteps])

x = np.array(x)
y = np.array(y)

### Split data

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Implementation of Model

### Define Model class

In [13]:
# reshape the input data
input_shape = (x_train.shape[1], x_train.shape[2])

# define the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=input_shape, return_sequences=True))
model.add(Dense(1, activation='linear')) # TODO:Adjust the activation function to sigmoid if needed

### Compile model

In [14]:
# compile the model for training
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae']) # TODO:Adjust the loss function to binary_crossentropy if needed

### Train and validate model

In [16]:

# # train the model
# model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))


# Custom callback to save and print metrics at each epoch
class EpochLogger(Callback):
    def __init__(self):
        super().__init__()
        self.epochs = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        # Print the metrics for the current epoch
        print(f"Epoch {epoch + 1}: {logs}")

        # Save metrics in a list for later use
        self.epochs.append({"epoch": epoch + 1, **logs})

# Instantiate the custom callback
epoch_logger = EpochLogger()

# Train the model and use the custom callback
history = model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[epoch_logger]
)

# Save the epoch logs to a CSV file
epoch_logs_df = pd.DataFrame(epoch_logger.epochs)
epoch_logs_df.to_csv('epoch_logs.csv', index=False)

# Print the saved dataframe
print(epoch_logs_df)

Epoch 1/20
2495/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 6970.1860 - mae: 78.9376Epoch 1: {'loss': 3566.484130859375, 'mae': 51.41018295288086, 'val_loss': 171.22882080078125, 'val_mae': 10.754624366760254}
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 27s 9ms/step - loss: 6962.0254 - mae: 78.8716 - val_loss: 171.2288 - val_mae: 10.7546
Epoch 2/20
2494/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 99.6329 - mae: 6.1039Epoch 2: {'loss': 78.4128646850586, 'mae': 4.918132305145264, 'val_loss': 66.81493377685547, 'val_mae': 4.675384521484375}
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 24s 10ms/step - loss: 99.5735 - mae: 6.1006 - val_loss: 66.8149 - val_mae: 4.6754
Epoch 3/20
2498/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 66.6841 - mae: 4.6726Epoch 3: {'loss': 66.51160430908203, 'mae': 4.666720390319824, 'val_loss': 66.05868530273438, 'val_mae': 4.649550914764404}
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step - loss: 66.6839 - mae: 4.6726 - val_loss: 66.0587 - val_mae: 4.6496
Epoch 4/20
2496/2500 ━━━━━━━━━━

### Test model

In [ ]:
# Predict on the test data
y_pred = model.predict(x_test)

# Inverse transform the predictions
y_pred_original = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))

## Other resources code

### Load model

In [ ]:
loaded_model = load_model(Save_model_dir)

### Save model

In [ ]:
# save in tensoflow model
model.save(Save_model_dir)